In [1]:
import sys
sys.path.append('..')
from osp import *

In [3]:
df = get_jstor_data()

In [4]:
df.decade.value_counts()

decade
2010    5346
2000    3736
1980    3481
1970    3249
1990    3136
1950    2634
1960    2629
2020    2018
1940    1901
1930    1726
1920    1147
1910     665
1900     609
1890     480
1880      26
Name: count, dtype: int64

In [ ]:
def iter_jstor_sample():
    df = get_jstor_data().sample(frac=1)
    decs = list(df.groupby('decade'))
    maxn = max(len(decdf) for dec,decdf in decs)
    for i in range(maxn):
        for dec,decdf in decs:
            if len(decdf) > i:
                yield decdf.iloc[i]

In [6]:
row = next(iter_jstor_sample())

In [10]:
row

id                                    0926057b-a45e-37e6-a9b5-959be1ebc485
review_required                                                       True
ithaka_doi                                                 10.2307/4543555
identifiers              {'print_isbn': None, 'online_isbn': None, 'pri...
title                                                               Beauty
is_part_of                         Proceedings of the Aristotelian Society
creators_string                                              P. N. Waggett
creators                 [{'first_name': 'P. N.', 'last_name': 'Waggett...
publishers               [The Aristotelian Society, Oxford University P...
published_date                                                   1889-01-0
languages                                                            [eng]
discipline_names                                              [Philosophy]
issue_number                                                             3
issue_volume             

In [11]:
def parse_jstor_row(row, force=False):
    id = row['id']
    if not force and id in JSTOR_HASH:
        return stanza.Document.from_serialized(JSTOR_HASH[id])
    
    text = row['full_text']
    article = filter_pmla_pages(text)
    doc = get_nlp_doc(article)
    JSTOR_HASH[id] = doc.to_serialized()
    return doc

In [12]:
doc = parse_jstor_row(row)
tree_stats = pd.DataFrame(get_tree_stats(sent) for sent in doc.sentences)
tree_stats

,num_words,height,num_independent_clauses,num_dependent_clauses,num_words_in_dependent_clauses,num_words_in_independent_clauses,num_punct,num_punct_colon,num_punct_comma,num_parens
0,6,5,0,0,0,6,1,0,0,0
1,28,16,1,3,21,7,1,0,0,0
2,20,15,0,2,21,-1,2,0,1,0
3,49,17,3,4,57,-8,6,0,5,0
4,23,11,2,0,0,23,3,0,2,0
...,...,...,...,...,...,...,...,...,...,...
261,40,19,2,3,51,-11,5,0,4,0
262,41,17,3,2,46,-5,4,0,3,0
263,25,15,0,1,25,0,2,0,1,0
264,54,24,3,2,36,18,5,0,4,0


In [13]:
tree_stats.mean()

num_words                           27.116541
height                              12.947368
num_independent_clauses              1.605263
num_dependent_clauses                1.206767
num_words_in_dependent_clauses      16.251880
num_words_in_independent_clauses    10.864662
num_punct                            3.609023
num_punct_colon                      0.101504
num_punct_comma                      2.368421
num_parens                           0.120301
dtype: float64

In [14]:
sample = list(iter_jstor_sample())

In [ ]:
for row in tqdm(sample):
    parse_jstor_row(row)

  0%|          | 14/32783 [08:00<454:28:54, 49.93s/it]